In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [30]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [31]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMZN', 'ANET',
    'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'DDOG', 'DOCS', 'DOCU',
    'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED', 'GOOG', 'INTG',
    'ISRG', 'JD', 'LSPD', 'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH',
    'NVCR', 'NVDA', 'OKTA', 'OPEN', 'PANW', 'PAYC', 'PINS', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP',
    'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO',
    'TWTR', 'UPST', 'VEEV', 'VLD', 'XPEV', 'ZEN', 'ZM']

In [32]:
!ls $DL

Portfolio_Positions_Jul-29-2022.csv clean.csv
Positions.csv


In [33]:
!head -5 $DL/Positions.csv

"Account 54253759 Positions, as of 07/29/22 at 07:18 PM EST"
"Position","Qty","Mark","Bid","Ask","Market Value","Today's Market Gain %"
"AAPL shares","4","162.51","162.16","162.19","650.04","3.279313632030503"
"ABMD shares","3","293.01","215.0","310.0","879.03","0.27720739219711954"
"ABNB shares","4","110.98","110.8","110.95","443.92","1.9661889011392875"


In [34]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [35]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [36]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [37]:
DONGMEI = False

In [38]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,650.04
ABMD,879.03
ABNB,443.92
ADYEY,5586.2
AMD,2078.34


In [39]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,650
ABMD,879
ABNB,444
ADYEY,5586
AMD,2078


In [40]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [45]:
et_df.loc[et_df.ET == '--']
#et_df.loc['GH', 'ET'] = 50.17

,ET


In [46]:
check_symbols(et_df)

All clear


In [47]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,650
ABMD,879
ABNB,444
ADBE,0
ADYEY,5586


### Fidelity

In [48]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Jul-29-2022.csv'

In [49]:
!ls $DL

Portfolio_Positions_Jul-29-2022.csv clean.csv
Positions.csv


In [50]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("06", "07")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,165.61,$1.00,$0.00,$165.61,NaN,NaN,NaN,NaN,0.43%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,10.00,$162.51,+$5.16,$1625.10,+$51.60,+3.27%,+$110.00,+7.26%,4.19%,$1515.10,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$293.01,+$0.81,$1172.04,+$3.24,+0.27%,+$153.35,+15.05%,3.02%,$1018.69,$254.67,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$110.98,+$2.14,$221.96,+$4.28,+1.96%,-$63.54,-22.26%,0.57%,$285.50,$142.75,Cash
4,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,1.00,$18.02,+$0.705,$18.02,+$0.70,+4.07%,+$2.93,+19.41%,0.05%,$15.09,$15.09,Cash


In [51]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,1625
2,ROLLOVER IRA,ABMD,1172
3,ROLLOVER IRA,ABNB,222
4,ROLLOVER IRA,ADYEY,18
5,ROLLOVER IRA,AMD,1512


In [52]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,1625
ABMD,1172
ABNB,222
ADYEY,18
AMD,1512


In [53]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [54]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,650,1625,9426,0
ABMD,879,1172,2344,0
ABNB,444,222,333,0
ADBE,0,0,0,0
ADYEY,5586,18,4667,0


In [55]:
out.to_csv(f'{DL}/clean.csv')

In [56]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [9]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [10]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_1,Value_2,Value_3
AAPL,708,425,0
ABMD,1021,766,0
ABNB,621,518,1035
ADYEY,615,1755,1335
AMD,784,1916,1306


In [11]:
check_symbols(mw)

All clear


In [12]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,708,425,0
ABMD,1021,766,0
ABNB,621,518,1035
ADYEY,615,1755,1335
AMD,784,1916,1306


In [13]:
mw.to_csv(f'{DL}/mw.csv')

In [14]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*